## Train Model

### 1. Cài đặt các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

### 2. Đọc dữ liệu từ file "cleaned_arxiv_cs_papers_all.csv"

In [2]:
file_name = 'cleaned_arxiv_cs_papers_all.csv'
raw_df = pd.read_csv(file_name)
raw_df.head()
raw_df.shape

(314049, 10)

In [3]:
raw_df.head()

,paper_id,authors,updated,published,title,abstract,categories,comment,topic_count,published_year
0,2412.10373v1,"Sicheng Zuo,Wenzhao Zheng,Yuanhui Huang,Jie Zh...",2024-12-13 18:59:54,2024-12-13 18:59:54,GaussianWorld: Gaussian World Model for Stream...,3D occupancy prediction is important for aut...,"cs.CV, cs.AI, cs.LG",Code is available at: https://github.com/zuosc...,3,2024
1,2412.10371v1,"Wenzhao Zheng,Junjie Wu,Yao Zheng,Sicheng Zuo,...",2024-12-13 18:59:30,2024-12-13 18:59:30,GaussianAD: Gaussian-Centric End-to-End Autono...,Vision-based autonomous driving shows great ...,"cs.CV, cs.AI, cs.LG, cs.RO",Code is available at: https://github.com/wzzhe...,4,2024
2,2402.01886v2,"Mark Beliaev,Ramtin Pedarsani",2024-12-13 18:59:14,2024-02-02 20:21:09,Inverse Reinforcement Learning by Estimating E...,"In Imitation Learning (IL), utilizing subopt...","cs.LG, cs.AI","11 pages, 4 figures, extended version of AAAI ...",2,2024
3,2412.10360v1,"Orr Zohar,Xiaohan Wang,Yann Dubois,Nikhil Meht...",2024-12-13 18:53:24,2024-12-13 18:53:24,Apollo: An Exploration of Video Understanding ...,Despite the rapid integration of video perce...,"cs.CV, cs.AI",https://apollo-lmms.github.io,2,2024
4,2412.10354v1,"Jean Kossaifi,Nikola Kovachki,Zongyi Li,Davit ...",2024-12-13 18:49:37,2024-12-13 18:49:37,A Library for Learning Neural Operators,"We present NeuralOperator, an open-source Py...","cs.LG, cs.AI",Not available,2,2024


#### Bổ sung các thư viện cho quá trình huấn luyện mô hình

In [ ]:
!pip install torch transformers scikit-learn tqdm

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

### 3. Chuyển đổi nhãn
- Ghép tiêu đề và tóm tắt của bài báo
- Chuyển nhãn thành dạng nhị phân

In [ ]:
# Cài đặt các tham số
MODEL_NAME = 'microsoft/deberta-v3-small'  # Mô hình DeBERTa
NUM_CLASSES = 20  # Số lớp phân loại (cần thay đổi theo số lượng danh mục bài báo của bạn)

# Tiền xử lý: Ghép tiêu đề và tóm tắt lại
raw_df['text'] = raw_df['title'] + " " + raw_df['abstract']

# Chuyển nhãn thành các giá trị nhị phân (ví dụ: cho nhiều nhãn)
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(raw_df['categories'].apply(lambda x: x.split(', ')))

In [ ]:
!pip install nltk

### 4. Tiền xử lý văn bản
- Chuyển văn bản về chữ thường.
- Loại bỏ ký tự đặc biệt và giữ lại chữ cái và khoảng trắng.
- Loại bỏ stopwords (các từ không mang nhiều ý nghĩa như 'the', 'is',...).
- Thực hiện lemmatization (đưa từ về dạng cơ bản, ví dụ: "running" -> "run").

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Tải các tài nguyên cần thiết từ NLTK
nltk.download('punkt')  # Để sử dụng word_tokenize
nltk.download('stopwords')  # Để sử dụng stopwords
nltk.download('wordnet')  # Để sử dụng lemmatizer


# Khởi tạo lemmatizer và stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Tiền xử lý dữ liệu
raw_df['text'] = raw_df['text'].str.lower()  # Chuyển về chữ thường
raw_df['text'] = raw_df['text'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))  # Loại bỏ ký tự đặc biệt
raw_df['text'] = raw_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))  # Loại bỏ stopwords
raw_df['text'] = raw_df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))  # Lemmatization


### 5. Chia dữ liệu thành tập huấn luyện và tập kiểm tra

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(raw_df['text'], labels, test_size=0.2)

### 6. Tokenize văn bản:
- Tải tokenizer của mô hình DeBERTa.
- Tokenize dữ liệu huấn luyện và kiểm tra:
    - Thêm padding để đảm bảo các chuỗi có cùng độ dài.
    - Cắt bớt chuỗi dài hơn giới hạn (max_length = 200).

In [ ]:
# Tải tokenizer của DeBERTa
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenize văn bản
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=200)

train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, max_length=200)
val_encodings = tokenizer(list(val_texts), padding=True, truncation=True, max_length=200)

### 7. Chuẩn bị DataLoader:
- Chuyển dữ liệu tokenized và nhãn thành TensorDataset để sử dụng với PyTorch.
- Tạo DataLoader cho tập huấn luyện và kiểm tra:
    - train_dataloader: Dùng để huấn luyện, với shuffle=True để xáo trộn dữ liệu.
    - val_dataloader: Dùng để kiểm tra mô hình.

In [ ]:
# Chuyển đổi dữ liệu thành Tensor
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_labels))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_labels))

# Tạo DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

In [ ]:
# Tải mô hình DeBERTa với số lượng nhãn được chỉ định (num_labels = NUM_CLASSES).
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASSES)

# Cấu hình optimizer AdamW để tối ưu hóa với AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

### 8. Huấn luyện và đánh giá mô hình:
- Thiết lập thiết bị (GPU nếu có, nếu không thì dùng CPU).
- Huấn luyện mô hình qua nhiều epoch:
    - Chạy mô hình trên tập huấn luyện (train_dataloader).
    - Tính toán loss với BCEWithLogitsLoss cho bài toán phân loại đa nhãn.
    - Thực hiện backward pass và cập nhật trọng số.
- Đánh giá trên tập kiểm tra (val_dataloader):
    - Sử dụng sigmoid để chuyển logits thành xác suất và so sánh với ngưỡng 0.
    - Tính toán accuracy dựa trên các dự đoán và nhãn thực tế.

In [ ]:
from sklearn.metrics import accuracy_score
from torch.nn import BCEWithLogitsLoss
# Huấn luyện mô hình
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Huấn luyện (Epochs)
epochs = 5
for epoch in range(epochs):
    model.train()
    loop = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}')
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        # Forward pass (Chạy mô hình)
        outputs = model(input_ids, labels=labels)
        logits = outputs.logits  # Lấy logits đầu ra từ mô hình

        # Sử dụng BCEWithLogitsLoss cho multi-label classification
        loss_fn = BCEWithLogitsLoss()
        loss = loss_fn(logits, labels.float())  # Chuyển labels sang float

        # Backward pass (Cập nhật trọng số mô hình)
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    # Đánh giá trên tập kiểm tra (validation)
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch[0].to(device)
            labels = batch[1].to(device)

            # Forward pass (Chạy mô hình)
            outputs = model(input_ids)
            logits = outputs.logits

            # Tính dự đoán (điểm số > 0.5 cho mỗi lớp)
            preds = torch.sigmoid(logits) > 0.5  # Các giá trị đầu ra >= 0.5 được coi là nhãn đúng

            # Cộng dồn các kết quả
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.numel()  # Tổng số nhãn

            all_labels.append(labels.cpu().numpy())  # Chuyển labels sang numpy để tính accuracy
            all_preds.append(preds.cpu().numpy())

    # Tính accuracy theo toàn bộ dataset
    accuracy = accuracy_score(np.concatenate(all_labels), np.concatenate(all_preds))
    print(f"Validation Accuracy: {accuracy:.4f}")

### 9. Lưu mô hình đã huấn luyện

In [ ]:
model.save_pretrained('./deberta_model')
tokenizer.save_pretrained('./deberta_model')

In [ ]:
%cd /content/deberta_model
!zip -r deberta_model.zip /content/deberta_model
from google.colab import files
files.download('deberta_model.zip')